# Ay119: Deep learning

In this assignment, you will apply your (newly acquired) knowledge of deep learning 101
to build a classifier for a real life problem (well, actually not, but almost
-- see the very end of this exercise).

**Prerequisites**:

Please watch [Dima's tutorial on deep learning from Astroinformatics-2019](https://www.youtube.com/watch?v=xP0EmroyOFg&)
and study the corresponding [Jupyter notebook](https://github.com/dmitryduev/braai/blob/master/nb/braai_train.ipynb)!

Run this in Google Colab! It has all that you will need.

<a href="https://colab.research.google.com/github/dmitryduev/ay119/blob/master/deep_learning/rbp.ipynb" target="_blank">
<img src="https://colab.research.google.com/assets/colab-badge.svg">
</a>

When using Colab, clone the repo and change the path to the data:

In [52]:
!git clone https://github.com/dmitryduev/ay119.git

fatal: destination path 'ay119' already exists and is not an empty directory.


In [53]:
path_base = '/content/'
path_data = '/content/ay119/deep_learning/data'

# path_base = './'
# path_data = './data'

## The problem: the `rbp` classifier for the `GTF`

 You are running a state-of-the-art *robotic* sky survey called the Geeky Transient Facility,
 or GTF, perhaps somewhere in the Southern Hemisphere.
 You have a fancy image-differencing pipeline that allows you to find interesting objects in the
 dynamic sky every night, such as supernovae, variable stars, and asteroids, and send them
 out to the world as alerts.

 However you notice two things:

 1. Your fancy image-differencing pipeline produces a lot of bogus detections.
 With the increasing data rates you are worried that one day you will run out of
 graduate students to examine/scan the potentially interesting candidate sources.

 2. Your AGI-based robot-telescope is extraordinarily sophisticated and smart,
 but has one major soft spot: it really likes platypuses. In fact, the robot likes them
 so much (obsessed, really) that whenever a platypus appears in the vicinity, it halts observing
 and instead takes a picture of it and sends it out as an alert.

So you decide to build a `real, bogus, or platypus` (`rbp` for short) classifier for GTF.


## Transfer learning

You are not the first one to encounter a problem like this, so you decide to apply the
[*transfer learning*](https://www.tensorflow.org/tutorials/images/transfer_learning)
technique and use the real-bogus classifier `braai` that is employed
by a similar survey called [ZTF](https://ztf.caltech.edu) running in the Northern Hemisphere.


Import the necessities:

In [76]:
from astropy.visualization import ZScaleInterval
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tqdm.keras import TqdmCallback
from keras.callbacks import EarlyStopping

### The model

Download a pre-trained model:

In [55]:
model_name = 'braai_d6_m9.h5'

if not (pathlib.Path(model_name)).exists():
    !wget https://github.com/dmitryduev/braai/raw/master/models/braai_d6_m9.h5

Load the model and examine its architecture:

In [56]:
model = tf.keras.models.load_model('braai_d6_m9.h5')

In [57]:
model.summary()

Model: "VGG6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 61, 61, 16)        448       
_________________________________________________________________
conv2 (Conv2D)               (None, 59, 59, 16)        2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 29, 29, 16)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 27, 27, 32)        4640      
_________________________________________________________________
conv4 (Conv2D)               (None, 25, 25, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0      

We will be using the [functional API of `tf.keras`](https://www.tensorflow.org/guide/keras/functional).

Next, remove the output layer, but leave the feature extraction part of the network in place:

In [58]:
# model_fe = tf.keras.Model(inputs=, outputs=)
model_fe = model
model_fe.pop()
model_fe.summary()

Model: "VGG6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 61, 61, 16)        448       
_________________________________________________________________
conv2 (Conv2D)               (None, 59, 59, 16)        2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 29, 29, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 29, 29, 16)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 27, 27, 32)        4640      
_________________________________________________________________
conv4 (Conv2D)               (None, 25, 25, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0      

Create a new output layer that will provide the probabilities for our three classes.
Hint: you will need 3 output neurons and the softmax activation function.

In [59]:
output = tf.keras.layers.Dense(3, )(model_fe.output)

In [60]:
model_tl = tf.keras.Model(inputs=model_fe.inputs, outputs=output)

The new model architecture is ready! You may optionally "freeze" the weights in the feature-extracting
part of the network and only train the "head" of the network that does the classification -- feel
free to play around with this once you're done with the exercise.

In [61]:
# mark layers as not trainable
# for layer in model_tl.layers[:-1]:
# 	layer.trainable = False

You can now compile the model and it is ready for training!

Hints:
- The [Adam optizimer](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)
  with the default settings is usually a good starting point
- You will need the [categorical crossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy) loss function
- You can try to use the standard [metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics) to track the performance such as
accuracy, precision, recall, and area under curve, or code your own one!

In [62]:
model_tl.compile(
  optimizer='Adam',
  loss='CategoricalCrossentropy',
  metrics=['accuracy']
)

### The data

You have collected training examples and now need to make a dataset out of them.

In [63]:
p = pathlib.Path(path_data)

bogus = np.array([np.load(pp) for pp in (p / 'bogus').glob('*.npy')])
real = np.array([np.load(pp) for pp in (p / 'real').glob('*.npy')])
platypus = np.array([np.load(pp) for pp in (p / 'platypus').glob('*.npy')])

data = np.vstack((bogus, real, platypus))

In [64]:
bogus.shape, real.shape, platypus.shape

((100, 63, 63, 3), (100, 63, 63, 3), (51, 63, 63, 3))

Make labels:

In [65]:
indices = [0 for _ in range(bogus.shape[0])] +\
    [1 for _ in range(real.shape[0])] +\
    [2 for _ in range(platypus.shape[0])]
labels = tf.one_hot(indices, depth=3, dtype=tf.int8).numpy()
# labels


Train/validation/test split (we will use an 81\% / 9\% / 10\% data split):

In [66]:
test_size = 0.1
val_size = 0.1
random_state = 42

train_indexes, test_indexes = train_test_split(range(len(labels)), shuffle=True,
                                               test_size=test_size, random_state=random_state)
train_indexes, val_indexes = train_test_split(train_indexes, shuffle=True,
                                              test_size=val_size, random_state=random_state)

In [67]:
batch_size = 4
shuffle_buffer_size = 4

epochs = 20

We will feed the model with data stored as `tf.data.Dataset`'s.

Hint: use the `tf.data.Dataset.from_tensor_slices` method.

In [69]:
train_dataset = tf.data.Dataset.from_tensor_slices((data[train_indexes], labels[train_indexes]))
val_dataset = tf.data.Dataset.from_tensor_slices((data[val_indexes], labels[val_indexes]))
test_dataset = tf.data.Dataset.from_tensor_slices((data[test_indexes], labels[test_indexes]))

Now shuffle, batch, and repeat the data. Let's also apply some data augmentation (random horizontal
and vertical flips) to make the classifier more robust:

In [71]:
# train_dataset = train_dataset.shuffle(shuffle_buffer_size).batch(batch_size).repeat(epochs)
# flipping:
train_dataset = train_dataset.cache().map(
    lambda image, label: (tf.image.random_flip_left_right(image), label)
).map(
    lambda image, label: (tf.image.random_flip_up_down(image), label)
).shuffle(shuffle_buffer_size).batch(batch_size).repeat(epochs)
val_dataset = val_dataset.batch(batch_size).repeat(epochs)
test_dataset = test_dataset.batch(batch_size)

We now need to compute for how many epochs to train, validate, and test the classifier:

In [72]:
steps_per_epoch_train = len(train_indexes) // batch_size - 1
steps_per_epoch_val = len(val_indexes) // batch_size - 1
steps_per_epoch_test = len(test_indexes) // batch_size - 1
steps_per_epoch_train, steps_per_epoch_val, steps_per_epoch_test

(49, 4, 5)

We don't want to [overfit](https://en.wikipedia.org/wiki/Overfitting),
so we will halt training if the loss on the validation dataset is not
improving for, say 5 consecutive epochs.

Hint: Look for a suitable callback in `tf.keras.callbacks` and
monitor `val_loss` and restore the best weights

Let's also make progress bars look nicer and use the `TqdmCallback` callback.

In [77]:
early_stopping_callback = EarlyStopping(
                              monitor="val_loss",
                              min_delta=1e-2,
                              patience=2,
                              verbose=1
                          )

tqdm_callback = TqdmCallback(verbose=1)

callbacks = [
    early_stopping_callback,
    tqdm_callback,
]


# callbacks = [
#     keras.callbacks.EarlyStopping(
#         # Stop training when `val_loss` is no longer improving
#         monitor="val_loss",
#         # "no longer improving" being defined as "no better than 1e-2 less"
#         min_delta=1e-2,
#         # "no longer improving" being further defined as "for at least 2 epochs"
#         patience=2,
#         verbose=1,
#     )
# ]


We are ready to fit the model!

Hint: you'll need the `fit` method. Set `verbose=0` for `TqdmCallback` to work properly.

In [102]:
h = model.fit(
    train_dataset,
    epochs=epochs,
    steps_per_epoch = steps_per_epoch_train,
    validation_steps = steps_per_epoch_val,
    # validation_data = val_dataset
    # callbacks=callbacks,
    # batch_size=batch_size,
    # epochs=epochs
    )

Epoch 1/20


ValueError: ignored

Great! The model is trained and we can now evaluate it on the test set:

In [103]:
train_dataset

<RepeatDataset shapes: ((None, 63, 63, 3), (None, 3)), types: (tf.float64, tf.int8)>

In [ ]:
model_tl.

Let's visualize our model's predictions:

In [ ]:
ll = {0: 'bogus', 1: 'real', 2: 'platypus'}

for ii in test_indexes:
    interval = ZScaleInterval()
    limits = interval.get_limits(data[ii, ..., 0])

    fig = plt.figure(figsize=(3, 3), dpi=100)
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(data[ii, ..., 0], vmin=limits[0], vmax=limits[1])

    p = np.argmax(model_tl.predict(np.expand_dims(data[ii], axis=0))[0])
    display(f"{ll[p]}:")

    plt.show()
    # break

## Unsolicited wisdom

Even though this may seem like some ridiculous nonsense, you've actually implemented a real-life
transfer learning pipeline:
you took an off-the-shelf production DL model, then using a part of it as a pre-trained feature extractor,
adapted its architecture to a different problem, and trained on data the original model never saw, achieving
very good results.
